In [7]:
!pip install transformers==4.37.0 tqdm==4.66.1 datasets==2.14.6

  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   -------------------------- ------------- 5.5/8.4 MB 33.4 MB/s eta 0:00:01
   ---------------------------------------- 8.4/8.4 MB 28.9 MB/s eta 0:00:00
Using cached huggingface_hub-0.33.0-py3-none-any.whl (514 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 59.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.5
    Uninstalling tqdm-4.66.5:
      Successfully uninstalle

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2024.6.1 requires fsspec==2024.6.1.*, but you have fsspec 2023.10.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
from transformers import (
    AutoTokenizer,
    XLMRobertaForSequenceClassification, 
    TrainingArguments, 
    Trainer
    )

In [4]:
import json

with open('data/filter_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data

[{'content': 'Liệu các mô hình ngôn ngữ lớn có thể đạt đến mức độ hiểu biết và sáng tạo tương đương con người không, và những rủi ro đạo đức nào cần được xem xét?',
  'folder': 'technology',
  'topic': 'Trí tuệ nhân tạo'},
 {'content': 'Mạng nơ-ron tích chập (Convolutional Neural Networks - CNN) hoạt động như thế nào trong việc xử lý ảnh, và những cải tiến mới nhất trong kiến trúc CNN là gì?',
  'folder': 'technology',
  'topic': 'Deep Learning'},
 {'content': 'Thuyết tương đối rộng của Einstein tiên đoán sự tồn tại của lỗ đen, vậy bằng chứng thực nghiệm nào đã xác nhận điều này và những hệ quả vật lý nào phát sinh từ đó?',
  'folder': 'science',
  'topic': 'Vật lý học'},
 {'content': 'Liệu các mô hình Deep Learning có thể giải quyết được bài toán P=NP không?',
  'folder': 'technology',
  'topic': 'Deep Learning'},
 {'content': 'Những đột phá nào trong vật lý học có khả năng thay đổi cách chúng ta hiểu về vũ trụ trong 5 năm tới?',
  'folder': 'science',
  'topic': 'Vật lý học'},
 {'con

In [5]:
# label encoder
label2id = {
    ("technology", "Trí tuệ nhân tạo"): 0,
    ("technology", "Deep Learning"): 1,
    ("science", "Vật lý học"): 2
}

id2label =  {v: k for k, v in label2id.items()}
train_label_ids = [label2id[(d['folder'], d['topic'])] for d in data]

In [9]:
# tokenzier
#tokenizer roberta using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
encodings = tokenizer(
    [d['content'] for d in data], 
    truncation=True, 
    padding=True, 
    max_length=512
)

c:\Users\hiep.tranvan\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hiep.tranvan\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
# dataset class
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(encodings, train_label_ids)

In [18]:
#define model
model = XLMRobertaForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base"
)

c:\Users\hiep.tranvan\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hiep.tranvan\.cache\huggingface\hub\models--FacebookAI--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

In [23]:
#training 
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=1,
    save_steps=10,
    learning_rate=2e-5,
    logging_dir='./logs',
    report_to="none"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)
trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
# dự đoán thử
test_text = "Học sâu là gì?"
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True, max_length=64)
outputs = model(**inputs)
pred_label_id = torch.argmax(outputs.logits, dim=1).item()
pred_folder, pred_topic = id2label[pred_label_id]
print(f"Folder: {pred_folder}, Topic: {pred_topic}")

In [14]:
import json

filepath = 'trash/query.json'
with open(filepath, 'r', encoding='utf-8') as f:
    data = json.load(f)
print(len(data))

7


In [16]:
data[0]['query']

'liệu ai có thể thay thế vai trò của giáo viên trong giáo dục'